# K-Nearest Neighbor Model Implementation


<img src="../images/KNN - Model for Group Assigment.jpg" alt="Alternative text" />

#### IMPORTS


In [20]:
import numpy as np
import pandas as pd
import sklearn.neighbors as skl_nb
import sklearn.model_selection as skl_ms
import matplotlib.pyplot as plt

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from sklearn.exceptions import DataConversionWarning
warnings.filterwarnings(action='ignore', category=DataConversionWarning)

import sys
sys.path.append("..")

from utils.loading_data  import load_to_df_from_csv, get_all_feature_combinations
from utils.knn_functions import find_best_k_with_misclassification_cv, model_iterator_cv, data_normalizer

#### STEP 1: LOADING DATA


In [21]:
# Loading the train.csv as the main dataset
data = load_to_df_from_csv("../data/train.csv")

# Column Transformation to lowercase and underscored spaces
data.columns = data.columns.str.replace(' ', '_')
data.columns = data.columns.str.replace('-', '_')
data.columns = data.columns.str.lower()

X = data.loc[:, data.columns != 'lead']
y = data.loc[:, data.columns == 'lead']

### Exploring data


In [22]:
data.shape

(1039, 14)

In [23]:
data.describe()


,number_words_female,total_words,number_of_words_lead,difference_in_words_lead_and_co_lead,number_of_male_actors,year,number_of_female_actors,number_words_male,gross,mean_age_male,mean_age_female,age_lead,age_co_lead
count,1039.000000,1039.000000,1039.000000,1039.000000,1039.000000,1039.000000,1039.000000,1039.000000,1039.000000,1039.000000,1039.000000,1039.000000,1039.000000
mean,2334.256015,11004.368624,4108.256978,2525.024062,7.767084,1999.862368,3.507218,4561.855630,111.149182,42.353766,35.929588,38.716073,35.486044
std,2157.216744,6817.397413,2981.251156,2498.747279,3.901439,10.406632,2.088526,3417.855987,151.761551,7.817110,8.957193,12.285902,12.046696
min,0.000000,1351.000000,318.000000,1.000000,1.000000,1939.000000,1.000000,0.000000,0.000000,19.000000,11.000000,11.000000,7.000000
25%,904.000000,6353.500000,2077.000000,814.500000,5.000000,1994.000000,2.000000,2139.500000,22.000000,37.480769,29.500000,30.000000,28.000000
50%,1711.000000,9147.000000,3297.000000,1834.000000,7.000000,2000.000000,3.000000,3824.000000,60.000000,42.600000,35.000000,38.000000,34.000000
75%,3030.500000,13966.500000,5227.000000,3364.000000,10.000000,2009.000000,5.000000,5887.500000,143.500000,47.333333,41.500000,46.000000,41.000000
max,17658.000000,67548.000000,28102.000000,25822.000000,29.000000,2015.000000,16.000000,31146.000000,1798.000000,71.000000,81.333333,81.000000,85.000000


### Normalization Check

In [24]:
data_normalizer(X).describe()

,number_words_female,total_words,number_of_words_lead,difference_in_words_lead_and_co_lead,number_of_male_actors,year,number_of_female_actors,number_words_male,gross,mean_age_male,mean_age_female,age_lead,age_co_lead
count,1039.000000,1039.000000,1039.000000,1039.000000,1039.000000,1039.000000,1039.000000,1039.000000,1039.000000,1039.000000,1039.000000,1039.000000,1039.000000
mean,0.132193,0.145828,0.136419,0.097751,0.241682,0.800821,0.167148,0.146467,0.061818,0.449111,0.354449,0.395944,0.365206
std,0.122167,0.102987,0.107301,0.096772,0.139337,0.136929,0.139235,0.109737,0.084406,0.150329,0.127353,0.175513,0.154445
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.051195,0.075570,0.063310,0.031505,0.142857,0.723684,0.066667,0.068693,0.012236,0.355399,0.263033,0.271429,0.269231
50%,0.096897,0.117770,0.107220,0.070989,0.214286,0.802632,0.133333,0.122777,0.033370,0.453846,0.341232,0.385714,0.346154
75%,0.171622,0.190575,0.176684,0.130243,0.321429,0.921053,0.266667,0.189029,0.079811,0.544872,0.433649,0.500000,0.435897
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [25]:
feature_combinations = get_all_feature_combinations(X.columns)

# 8191 if needed to run for all combinations
results = model_iterator_cv(data_normalizer(X), y, feature_combinations, 10)

1 OUT OF 150 ITERATIONS COMPLETED - 0.6666666666666666%
2 OUT OF 150 ITERATIONS COMPLETED - 1.3333333333333333%
3 OUT OF 150 ITERATIONS COMPLETED - 2.0%
4 OUT OF 150 ITERATIONS COMPLETED - 2.6666666666666665%
5 OUT OF 150 ITERATIONS COMPLETED - 3.3333333333333335%
6 OUT OF 150 ITERATIONS COMPLETED - 4.0%
7 OUT OF 150 ITERATIONS COMPLETED - 4.666666666666667%
8 OUT OF 150 ITERATIONS COMPLETED - 5.333333333333333%
9 OUT OF 150 ITERATIONS COMPLETED - 6.0%
10 OUT OF 150 ITERATIONS COMPLETED - 6.666666666666667%
11 OUT OF 150 ITERATIONS COMPLETED - 7.333333333333333%
12 OUT OF 150 ITERATIONS COMPLETED - 8.0%
13 OUT OF 150 ITERATIONS COMPLETED - 8.666666666666666%
14 OUT OF 150 ITERATIONS COMPLETED - 9.333333333333334%
15 OUT OF 150 ITERATIONS COMPLETED - 10.0%
16 OUT OF 150 ITERATIONS COMPLETED - 10.666666666666666%
17 OUT OF 150 ITERATIONS COMPLETED - 11.333333333333334%
18 OUT OF 150 ITERATIONS COMPLETED - 12.0%
19 OUT OF 150 ITERATIONS COMPLETED - 12.666666666666666%
20 OUT OF 150 ITERAT